### Introduction

This time the goal is to go through some basic Machine Learning practices. We will use data we have been working on so far.

### Goal

The goal for this exercise is to create a Machine Learning model that will predict total number of bike rentals on a given day.

Even though, there may be more interesting things to look for in the data we have, this will be good opportunity to go through the process and then everyone should be able to work on other predictions.

This doesn't really take into consideration which stations were the bikes rented from or returned to. It doesn't matter what the routes were, etc. We only care for the total number of rentals for a given day.

### Requirements

We need to enrich the data and then prepare it for ML training. When this is done, we can eventually execute the models and evaluate them.

In [ ]:
# Let's start with initial imports we definitelly need across the whole notebook
import pandas as pd
import numpy as np

In [ ]:
# Next, we load the data and have a glance at it
wyp = pd.read_csv('../../data/wyp_2018_prepared.csv')

In [ ]:
wyp.sample(5)

Our goal here is to predict the total (from all stations) number of renatals for a given day. You should think which columns are helpful for that, and which are not. We need to get rid of those that do not add any useful information for this particular task. 

Let's get rid of those that are pretty obvious first.

In [ ]:
wyp.columns

In [ ]:
wyp_ml = wyp.drop(columns = ['bike_num', 'start_time', 'end_time', 'departure', 'return',
                             'duration_sec', 'start_hour', 'start_minute', 'month_day',
                             'duration_min', 'duration_hour', 'route', 'dep_id', 'dep_lat', 'dep_lon',
                             'ret_id', 'ret_lat', 'ret_lon', 'distance'])

In [ ]:
wyp_ml.sample(5)

As you can see, we dropped quite a few columns. But this will be very simple use case for which the information in them is not relevant for us. You may want to double check on that though, we will see how further in the notebook.

The prediction should tell us how many rentals will be there for a given day, so to train a model, we need to feed it with that information from the past. In its current form, the dataframe doesn't really show it, we need to count the rentals and add it to our set.

In [ ]:
sums = wyp_ml.groupby(['start_day', 'start_month'])['which_day'].count()
sums = sums.to_frame()
sums = pd.DataFrame(sums.to_records())
sums = sums.rename({'which_day': 'count'}, axis=1)
sums.sample(5)

Since we have saved now count of rentals per day, we can transform our DF so that it has a single row per day with the rentals number included.

In [ ]:
print(len(wyp_ml))
wyp_ml_drooped = wyp_ml.drop_duplicates()
print(len(wyp_ml_drooped))

In [ ]:
wyp_ml_2 = wyp_ml_drooped.merge(sums, left_on=['start_day', 'start_month'], right_on=['start_day', 'start_month'], how='left')
wyp_ml_2.reset_index(drop=True, inplace=True)
wyp_ml_2.head(5)

One last thing that seems to might have significant impact on the rentals number is weather. There are many ways to get weather information and add it to our data frame. We're not going into any details at this point however, simply load the data provided in a file.

In [ ]:
weather = pd.read_csv('../../data/weather.csv', index_col = 0)

In [ ]:
weather.head(10)

In [ ]:
# Let's add it to our main dataframe

wyp_ml_3 = pd.merge(weather, wyp_ml_2, on='daynumber')
wyp_ml_3.reset_index(drop=True, inplace=True)
wyp_ml_3.sample(10)

In [ ]:
wyp_ml_3.shape

In [ ]:
#wyp_ml_3.to_csv('k_dane.csv', index=False)

#from sklearn.model_selection import train_test_split

#train_set_tmp, test_set_tmp = train_test_split(wyp_ml_3, test_size=0.2, random_state=42)

#train_set_tmp.to_csv('k_dane_train.csv', index=False)
#test_set_tmp.to_csv('k_dane_test.csv', index=False)

#wyp_ml_3 = train_set_tmp

In [ ]:
#wyp_ml_3 = pd.read_csv('k_dane_test.csv')

In [ ]:
wyp_ml_3.sample(5)

In [ ]:
# At this moment we should drop columns with duplicated data

wyp_ml_4 = wyp_ml_3.drop(columns = ['daynumber', 'which_day'])

Before running ML algorithms it is good idea to have some sort of a benchmark. We will use mean number of rentals.

We also need to select a performance measure.

A typical performance measure for regression problems is the Root Mean Square Error (RMSE). It gives an idea of how much error the system typically makes in its predictions, with a higher weight for large errors.

There may be other better measures, but for our case the RMSE is good enough.

In [ ]:
avg = np.mean(wyp_ml_4['count'])
print('Just using average={0} has RMSE of {1}'.format(avg, np.sqrt(np.mean((wyp_ml_4['count'] - avg)**2))))

In [ ]:
# To get a feel of your data, you can plot a histogram for each numerical value.
# You can see there some of the values are continuous and others seem to be categorical which we will
# further analyze in next steps.

%matplotlib inline
import matplotlib.pyplot as plt
wyp_ml_4.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# Using simple scatter you can analyze dependency between features and labels

wyp_ml_4.plot(kind='scatter', x='maxtemp', y='count')

### Creating a Test Set

Let's get 20% of our dataset and save it as a test set to validate the model against when it's ready.

We could simply split the sets randomly, but a better idea might be to make sure we have good representation of data in our test set. Random sampling should be fine as long as the dataset is large enough, in this case the dataset is very small and thus we need to help with the split a little. This is called stratified sampling. The dataset is divided into homogeneous groups called strata, the goal is so that the right number of instances is sampled from each stratum for our test set to be good representation of the overall data. More here: https://medium.com/@411.codebrain/train-test-split-vs-stratifiedshufflesplit-374c3dbdcc36

Before running any ML models, we can predict that rain levels is significant factor for bike rentals numbers. This may not be the most relevant factor, but for that one, the distribution in the overall data is very skewed which means in a small dataset, its meaning might be easily lost.

The level of rain is continuous numerical attribute we need to create some categories for.

Looking at the rain histogram, we can say having 4 categories should be enough. Each category should be large enough to include good number of instances.

In [ ]:
wyp_ml_4.rain.describe()

In [ ]:
wyp_ml_4.rain.hist()

We will have following categories:
```
1- 0
2- 0   >= 0.3
3- 0.3 >= 0.6
4- 0.6 >=
```

In [ ]:
wyp_ml_4['rain_cat'] = pd.cut(wyp_ml_4.rain, bins = [-1.0, 0.0, 0.3, 0.6, np.inf], labels = [1, 2, 3, 4])

In [ ]:
wyp_ml_4.rain_cat.hist()

We will use Scikit-Learn’s StratifiedShuffleSplit class to do stratified sampling based on the rain levels category:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=44)
for train_index, test_index in split.split(wyp_ml_4, wyp_ml_4.rain_cat):
    strat_train_set = wyp_ml_4.loc[train_index]
    strat_test_set = wyp_ml_4.loc[test_index]

Let's see how many observations are there per rain levels category:

In [ ]:
strat_test_set["rain_cat"].value_counts() / len(strat_test_set)

Now, this doesn't look too good, we have underrepresentation of two last categories. This is due to the fact, there is simply not enough data with observations that fits them. We need to change the categories:

```
1- 0
2- 0   >= 0.2
3- 0.2 >=
```

In [ ]:
wyp_ml_4['rain_cat'] = pd.cut(wyp_ml_4.rain, bins = [-1.0, 0.0, 0.2, np.inf], labels = [1, 2, 3])

In [ ]:
wyp_ml_4.rain_cat.hist()

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(wyp_ml_4, wyp_ml_4.rain_cat):
    strat_train_set = wyp_ml_4.loc[train_index]
    strat_test_set = wyp_ml_4.loc[test_index]

In [ ]:
strat_test_set["rain_cat"].value_counts() / len(strat_test_set)

It's not perfect, but quite better then it was before.

Now we can remove the rain_cat attribute to have our set in its original form

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("rain_cat", axis=1, inplace=True)

Another way of splitting the data is to do it by start_month column. You should check which offers better result.

In [ ]:
# wyp_ml_4.start_month.hist()

In [ ]:
# split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# for train_index, test_index in split.split(wyp_ml_4, wyp_ml_4.start_month):
#     strat_train_set = wyp_ml_4.loc[train_index]
#     strat_test_set = wyp_ml_4.loc[test_index]

In [ ]:
# strat_test_set["start_month"].value_counts() / len(strat_test_set)

At this point you may want to run few more visualizations on your data. Let's check how attributes correlate to each other, but first reminder of Pearson Correlation.

![Example](../../data/mlst_0214.png)

In [ ]:
corr_matrix = wyp_ml_4.corr()

In [ ]:
corr_matrix["meantemp"].sort_values(ascending=False)

In [ ]:
import seaborn as sns

plt.figure(figsize = (15, 15))
sns.heatmap(corr_matrix, annot = True)

You should further examine most promising correlations. This step, in general, is used to find out oddities in your data. Sometimes values may be capped for instance and plotting correlation scatter will reveal this. 

Other case could involve attributes with tail-heavy distribution for which you may want to transform them using their logharithm instead for instance. 

In our case let's move on. 

At some point having dedicated meeting about visualizations and quirks they can reveal could be good idea, though.

### Data Preparation for ML Algorithms

Next step is to create functions to transform the data in a way that is best suited for ML algorithms. We will start with dividing the train set to predictors and labels.

In [ ]:
bikes = strat_train_set.drop("count", axis=1)
bikes_labels = strat_train_set["count"].copy()

In [ ]:
#bikes = wyp_ml_4.drop("count", axis=1)
#bikes_labels = wyp_ml_4["count"].copy()

Let's have a look at the code below and we will describe what it does once it is ran

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

cat_attribs = ['dayofweek', 'fog', 'start_day', 'start_month', 'is_weekend', 'is_free', 'shop']
num_attribs = ['mintemp', 'maxtemp', 'rain', 'meantemp', 'dewp', 'visib', 'mxpsd', 'sndp']

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

First of all, we need to know which attributes are numerical and which are categorical. 

Then we may want to apply transformations on each of them separately or on all attributes altogether.

In our case, num_pipeline includes transformations that will be applied on numerical attributes only. It will look for missing values and replace them with the median for the given feature. Then it will standardise all numerical attributes.

*** What do you think should be the approach here, should be standardise, normalize (min-max scaling) values or maybe a mix of both?

We have a single transformation to be applied on categorical attributes, it is one hot encoder.

In [ ]:
bikes_prepared = full_pipeline.fit_transform(bikes)

*** Remember, it is important to fir scalers to the train data only, not the full dataset or the test dataset. This is especially important so that you can easily add new data for instance. 

In [ ]:
pd.DataFrame(bikes_prepared.toarray())

### Choosing and Training a Model!

So now it is the time for what we all waited for ... let's get on with the most interesting and spectacular part - training a Machine Learning Model!!!

We will start with Linear Regression.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(bikes_prepared, bikes_labels)

In [ ]:
from IPython.display import HTML
HTML('<img src="https://media.giphy.com/media/LwHfaQM6AzEu44CqJO/giphy.gif">')

Well ... that wasn't spectacular at all.

But that indeed is what we need to train a model. Anyway, let's have a look at how it went.

In [ ]:
from sklearn.metrics import mean_squared_error
bikes_predictions = lin_reg.predict(bikes_prepared)
lin_mse = mean_squared_error(bikes_labels, bikes_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

Not too bad - compared to our benchmark which was 340! But we can do better I'm sure. Decision Trees next

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(bikes_prepared, bikes_labels)

In [ ]:
bikes_predictions = tree_reg.predict(bikes_prepared)
tree_mse = mean_squared_error(bikes_labels, bikes_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

Hm ... Is this for real? No error at all? Do we have a perfect model? Well ... no. This is an example of overfitting the data, badly. How to verify this? Until you know which model to choose, you shouldn't touch the test data. Instead, you should use part of the training set for training and part for model validation.

One way to do it is by using the Scikit-Learn's cross-validation feature. It randomly splits the dataset into a number of distinct subsets called folds, then it trains and evaluates the Decision Trees model as many times as many folds there are using different fold for evaluation every time and training on the remaining 9 folds. We have an array of 10 evaluation scores as the result.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, bikes_prepared, bikes_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
display_scores(tree_rmse_scores)

Aaaand it doesn't look that good any more. In fact it seems to perform a lot worse than simple linear regression model. 

With cross-validation you also get standard deviation - a measure of how precise the estimate is. So the Decision Tree has a score of 183 +/- 37.

Let's check the score the same way for Linear Regression, though

In [ ]:
lin_scores = cross_val_score(lin_reg, bikes_prepared, bikes_labels,
                             scoring="neg_mean_squared_error", cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Indeed, it seems that the Decision Tree is overfitting too hard and as a result it performs worse then a Linear Regression model. 

Let's try Random Forest now.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(bikes_prepared, bikes_labels)

In [ ]:
bikes_predictions = forest_reg.predict(bikes_prepared)
forest_mse = mean_squared_error(bikes_labels, bikes_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
forest_scores = cross_val_score(forest_reg, bikes_prepared, bikes_labels,
                             scoring="neg_mean_squared_error", cv=10)

forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Now, this is even better. Bear in mind that the score on the training set is still significantly lower than on the validation sets. This means the model is still overfitting the training set.

XGBoost as the last one

In [ ]:
import xgboost as xgb

xgboost_reg = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
xgboost_reg.fit(bikes_prepared, bikes_labels)

In [ ]:
bikes_predictions = xgboost_reg.predict(bikes_prepared)
xgboost_mse = mean_squared_error(bikes_labels, bikes_predictions)
xgboost_rmse = np.sqrt(xgboost_mse)
xgboost_rmse

In [ ]:
xgboost_scores = cross_val_score(xgboost_reg, bikes_prepared, bikes_labels,
                             scoring="neg_mean_squared_error", cv=10)

xgboost_rmse_scores = np.sqrt(-xgboost_scores)
display_scores(xgboost_rmse_scores)

### Fine tunning the models

#### Linear Regression

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_lr = [
    {'fit_intercept':[True], 'normalize':[True,False], 'copy_X':[True,False]},
    {'fit_intercept':[False], 'copy_X':[True,False]},
]
    
linear_reg = LinearRegression()
grid_search_lr = GridSearchCV(linear_reg, param_grid_lr, cv=10,
                           scoring='neg_mean_squared_error', return_train_score=True)

grid_search_lr.fit(bikes_prepared, bikes_labels)

In [ ]:
grid_search_lr.best_params_

In [ ]:
cvres_lr = grid_search_lr.cv_results_

for mean_score, params in zip(cvres_lr["mean_test_score"], cvres_lr["params"]):
    print(np.sqrt(-mean_score), params)

#### Random Forest

In [ ]:
param_grid_rf = [
    {'n_estimators': [3, 10, 100, 1000], 'max_features': [4, 16, 64]},
    {'bootstrap': [False], 'n_estimators': [3, 10, 100, 1000], 'max_features': [4, 16, 64]},
  ]

forest_reg = RandomForestRegressor()

grid_search_rf = GridSearchCV(forest_reg, param_grid_rf, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search_rf.fit(bikes_prepared, bikes_labels)

In [ ]:
grid_search_rf.best_params_

In [ ]:
cvres_rf = grid_search_rf.cv_results_

for mean_score, params in zip(cvres_rf["mean_test_score"], cvres_rf["params"]):
    print(np.sqrt(-mean_score), params)

### Evaluating on the Test Set

In [ ]:
final_model = grid_search_lr.best_estimator_

X_test = strat_test_set.drop("count", axis=1)
y_test = strat_test_set["count"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

In [ ]:
final_model = grid_search_rf.best_estimator_

X_test = strat_test_set.drop("count", axis=1)
y_test = strat_test_set["count"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

### Summary

We have defined our goal, prepared the dataset with data that is significant to predict what we're looking for. Then, we have transformed the data so that it is in a form that is readable by Machine Learning algorithms. 

Eventually, we have trained a few different models and evaluated their performance.

This doesn't cover every aspect of a Machine Learning project yet, though. We haven't touched fine tuning yet for instance. We should get back to it next time.